
#            Calcul numérique du MGI d'un RRR par PNL #



On se propose de calculer de manière numérique un modèle géométrique inverse (MGI) d'un robot RRR en utilisant les outils de programmation non-linéaire. 

La configuration du robot est défini par le vecteur $q = (q_1, q_2, q_3)^t$.

La situation de l'outil est défini par le vecteur $X = (x, y, z)^t$ .

|![alternative text](robot-rrr3.png)|
|:--:| 
| *  Robot RRR  * |

## Objectif
L'objectif est d'implémenter plusieurs méthodes de recherche de solution pour le calcul du MGI (recherche d'un optimum local), de les comparer et d'en tirer des conclusions. 

Les conditions de sortie des algorithmes seront définies par un nombre
maximum d'itérations et par une valeur seuil pour l'erreur de la fonction objectif.


## Principe

Pour calculer le MGI avec les outils de programmation non linéaire vous devez :

- définir une fonction à minimiser ou à annuler

- implémenter une méthode de recherche linéaire


## Environnement Python et liste des modules utiles


In [ ]:
## Environnement Python
################################################# 
import numpy as np
import scipy
import time
##############
%matplotlib inline
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d  # Fonction pour la 3D
from mpl_toolkits.mplot3d import proj3d
#############
from scipy.optimize import minimize
from scipy.optimize import root
import scipy.optimize as spop
from scipy import linalg


## Calcul du MGD


In [ ]:
#################################################   
# Calcul du MGD du robot RRR
# INPUT:  q = vecteur de configuration (deg, deg, deg)
# OUTPUT: Xc = vecteur de situation = position3D = (x,y,z) 
def mgd(qdeg,a,b):
    qrad=np.deg2rad(qdeg)
    c1= np.cos(qrad[0])
    s1=np.sin(qrad[0])
    c23= np.cos(qrad[2]+qrad[1])
    s23= np.sin(qrad[2]+qrad[1])
    c2=np.cos(qrad[1])
    s2=np.sin(qrad[1])
    x= a*c1*c2 + b*c1*c23
    y= a*s1*c2 + b*s1*c23
    z= a*s2 + b*s23
    Xd=[x,y,z]
    return Xd

### Test du MGD

In [5]:
##### INPUT de q en degré ###
qdeg = [90, 0, 90]
Xd= mgd(qdeg,1,1)
print("X=", Xd[0], "Y = ", Xd[1], "Z= ",Xd[2])

X= 6.123233995736766e-17 Y =  1.0 Z=  1.0


## Calcul de la Jacobienne analytique Ja(q)

In [6]:
#################################################   
# Calcul de J(q) du robot RRR
# INPUT:  q = vecteur de configuration (deg, deg, deg)
# OUTPUT: jacobienne(q) analytique:
def ja(qdeg,a,b):
#### Paramètres du robot
    a,b=10,10
    qrad=np.deg2rad(qdeg)
    c1= np.cos(qrad[0])
    s1=np.sin(qrad[0])
    c23= np.cos(qrad[2]+qrad[1])
    s23= np.sin(qrad[2]+qrad[1])
    c2=np.cos(qrad[1])
    s2=np.sin(qrad[1])
 
    Ja=np.array([[-a*s1*c2 -b*s1*c23, -a*c1*s2 -b*c1*s23,  -b*c1*s23], 
                [a*c1*c2 + b*c1*c23, -a*s1*s2 -b*s1*s23,  -b*s1*s23], 
                 [0, a*c2 + b*c23, b*c23]])

    return Ja

### Test de J

In [7]:
qdeg = [90, 0, 90]
Ja=ja(qdeg,1,1)
print("Ja=",Ja)

Ja= [[-1.000000e+01 -6.123234e-16 -6.123234e-16]
 [ 6.123234e-16 -1.000000e+01 -1.000000e+01]
 [ 0.000000e+00  1.000000e+01  6.123234e-16]]


### Dessin du robot en 3D (filaire)

In [6]:
def dessinRRR(q,a,b) :
    qrad=np.deg2rad(q)
    c1= np.cos(qrad[0])
    s1=np.sin(qrad[0])
    c23= np.cos(qrad[2]+qrad[1])
    s23= np.sin(qrad[2]+qrad[1])
    c2=np.cos(qrad[1])
    s2=np.sin(qrad[1])
    
    X=[0, a*c1*c2, a*c1*c2 + b*c1*c23]
    Y=[0, a*s1*c2, a*s1*c2 + b*s1*c23]
    Z=[0, a*s2, a*s2 + b*s23]
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')  # Affichage en 3D
    ax.plot3D(X,Y,Z, 'red')
    
    plt.title("Tracé filaire")
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.scatter(X,Y,Z, c="red")
    plt.show()

### Affichage du robot en 3D

In [7]:
%matplotlib qt
dessinRRR(qdeg,5,5)

In [8]:
%matplotlib inline

# ===========================================

# Travail à faire

Vous devez définir et implémenter la fonction de calcul du mgi: 

$[q_{sol}]= mgi(X_d, q_0, Nbmax, \epsilon$) 

avec 

- $X_d$ situation désirée, 
- $q_0$ configuration initiale du robot,
- $Nbmax$ nombre maximum d'itérations, 
- $\epsilon$ précision du calcul de la solution.

en utilisant une des méthodes proposées.

## 1 - Méthode du gradient

On utilise la méthode du gradient pour minimiser un critère $C(q)= \frac{1}{2} \; e^t.e = \frac{1}{2} \; \left\lVert e \right\rVert^{2}$ avec $e= X_d -f(q)$, $f(q)$ correspond au MGD en $q$.

- Montrer que $direction = -\nabla C(q) = J(q)^t . e$.

- Implémenter la méthode du gradient pour résoudre le  MGI. Tester pour différentes conditions initiales (proches/loin de la solution, proches/loin de singularités. On choisira un coefficient $pas = 0.5$, puis on le fera varier. Quel coefficient semble le plus approprié ? 
- Tracer la variation de l'erreur. Comment régler le pas afin de garantir une convergence du gradient ? 
- Proposer et implémenter une solution de calcul automatique du pas.
- Modifer la longueur des corps (a = 1 ou 10, b = 1 ou 10). Avez-vous le même résultat ?
- Conclusions

In [8]:
###################################################################################
# Afin de donner une situation atteignable pour le robot,
# vous pouvez utiliser le mgd pour définir Xbut à partir d'une configuration en q
###################################################################################
## qbut est donné en degré
qbut= [0., 0,  -90.]
## Calcul Xbut à partir de qbut
Xbut= np.asarray(mgd(qbut,1,1))
print("Xbut=", Xbut[0], "Ybut = ", Xbut[1], "Z but = ", Xbut[2])

Xbut= 1.0 Ybut =  0.0 Z but =  -1.0


In [10]:
########### A FAIRE ##################################  
def direction():
   
    return dir

In [12]:
########## A FAIRE ###########################
def pas():
   
    return p

In [ ]:
######### CALCUL DU MGI A FAIRE ################
# Définir les paramètres du robot
a,b= 1,1
# Définition de Xbut
qbut= np.asarray([118., 22,  48.])
Xbut= np.asarray(mgd(qbut,a,b))
print("Xbut=", Xbut, type(Xbut))
#Initialisation de la recherche
qinit=np.asarray([120., 25,  45.])
Xinit=np.asarray(mgd(qinit,a,b))
print("Xinit = ",Xinit)
#
# Algo de recherche de la solution
# while (???):
#
#   A COMPLETER
#
qc = ??? #valeur finale retournée par MGI
# Visualisation des résultats (quelques idées, faire une courbe 
# qui montre l'évolution de l'erreur)
print("Valeur finale du critère =X",erx," après ",i," itérations")
X= mgd(qc,a,b)
print("Xinit              =",Xinit))
print("Xfinal avec qfinal = ",X)
print("Xbut à atteindre   =", Xbut)

print("Erreur finale=",erx)
abs = np.linspace(0,len(list_erreur)-1,(len(list_erreur)))

plt.plot(abs,list_erreur,'k')
plt.show(block=True)


## 2 - Méthode de Newton

On utilise le schéma de Newton pour calculer le MGI du robot en cherchant le zéro de la fonction:

$H(q) = X_d - f(q)$ 

pour une situation désirée $X_d = (x, y,\theta)^t$ avec $f(q)$ qui correspond au MGD en $q$.
Ce schéma impose de calculer l'inverse de la jacobienne analytique.

- Montrer que $ direction = J(q)^{-1} . (X_d - f(q)) $.
- Implémenter la méthode de Newton en python pour résoudre le MGI. Tester pour différentes conditions initiales(proches/loin de la solution, proches/loin de singularités. Modifier la valeur du $pas$. Faire varier  $N_{max}$ et $\epsilon$.
- Tracer la variation de l'erreur. Conclusions.


In [ ]:
################################################  
# calcul de la direction

In [ ]:
#################################################
# calcul du pas

In [ ]:
######### CALCUL DU MGI A FAIRE ################

# Conclusion

Donner vos conclusion sur l'utilisation de techniques de PNL pour le calcul de solutions de MGI

## 3 - Génération de mouvement en ligne droite

Réaliser un programme qui utilise le calcul numérique du MGI pour faire déplacer le robot en ligne droite entre deux points $A$ et $B$ définis dans l'espce de travail 3D.

A la situation initiale on connaît toujours la configuration $\bf{q}_0$ du robot (lecture des valeurs des codeurs).

Il faudra échantillonner le segment $AB$ pour définir les situations intermédiaire à atteindre.

Donner vos conclusions en fonction des points $A$ et $B$ et de la valeur de votre échantillonnage.


In [ ]:
plt.close('all')